In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
X = train.drop('class', axis=1)
y = train['class']
X_test = test

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns=test.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=test.columns)

C:\Users\hungu\anaconda.x\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\hungu\anaconda.x\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\hungu\anaconda.x\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [4]:
X.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5
0,-0.827565,0.827565,0.591354,-0.777115,-0.811333,-0.883497,-0.401060,-0.401011,-0.381579,-0.381681,-0.298352,-0.344407,2.016721,-0.191419,-0.16967
1,1.208365,-1.208365,-0.190164,-0.433562,1.105034,1.075244,0.404473,0.404769,-0.410242,-0.410316,3.351751,-0.344407,-0.495854,-0.191419,-0.16967
2,1.208365,-1.208365,-0.141319,-0.252745,0.733259,0.719473,-0.349742,-0.350704,-0.417798,-0.417895,-0.298352,-0.344407,-0.495854,-0.191419,-0.16967
3,-0.827565,0.827565,-0.727458,-0.795196,-1.008719,-0.787559,0.025582,0.025578,1.276687,1.276759,-0.298352,-0.344407,2.016721,-0.191419,-0.16967
4,1.208365,-1.208365,0.005215,-0.180418,-0.201928,-0.295875,-0.321791,-0.322674,-0.430566,-0.430589,-0.298352,-0.344407,-0.495854,-0.191419,-0.16967


In [5]:
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss, accuracy_score

err = []
y_pred_tot_lgm = []
features = [col for col in X.columns if col not in ["class"]]

from sklearn.model_selection import StratifiedKFold, KFold
feature_importance_df = pd.DataFrame()

fold = StratifiedKFold(n_splits=20, shuffle=True)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMClassifier(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=500,
          eval_metric='log_loss',
          verbose=200)
    pred_y = m.predict_proba(x_val)
    print("err_lgm: ",log_loss(y_val,pred_y))
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    err.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.204293	valid_1's binary_logloss: 0.380299
[400]	training's binary_logloss: 0.120046	valid_1's binary_logloss: 0.393474
[600]	training's binary_logloss: 0.0747271	valid_1's binary_logloss: 0.41364
Early stopping, best iteration is:
[156]	training's binary_logloss: 0.225778	valid_1's binary_logloss: 0.375042
err_lgm:  0.3750421364700357
Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.207532	valid_1's binary_logloss: 0.293169
[400]	training's binary_logloss: 0.124991	valid_1's binary_logloss: 0.323871
[600]	training's binary_logloss: 0.075598	valid_1's binary_logloss: 0.365965
Early stopping, best iteration is:
[173]	training's binary_logloss: 0.220419	valid_1's binary_logloss: 0.289125
err_lgm:  0.289125490019092
Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.210462	valid_1's binary_logloss: 

[600]	training's binary_logloss: 0.0779751	valid_1's binary_logloss: 0.271239
[800]	training's binary_logloss: 0.0470474	valid_1's binary_logloss: 0.286532
Early stopping, best iteration is:
[358]	training's binary_logloss: 0.142904	valid_1's binary_logloss: 0.249097
err_lgm:  0.2490969308874498
Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.212742	valid_1's binary_logloss: 0.223959
[400]	training's binary_logloss: 0.126415	valid_1's binary_logloss: 0.219704
[600]	training's binary_logloss: 0.0769353	valid_1's binary_logloss: 0.21887
Early stopping, best iteration is:
[266]	training's binary_logloss: 0.181591	valid_1's binary_logloss: 0.209787
err_lgm:  0.2097866023994563
Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.210195	valid_1's binary_logloss: 0.22848
[400]	training's binary_logloss: 0.130955	valid_1's binary_logloss: 0.220777
[600]	training's binary_logloss: 0.0794487	valid_1's

In [6]:
np.mean(err,0)

0.2721625081370983

In [7]:
all_feat = feature_importance_df[["Feature", 
                                      "importance"]].groupby("Feature").mean().sort_values(by="importance", 
                                                                                           ascending=False)
all_feat.reset_index(inplace=True)
important_feat = list(all_feat['Feature'])
all_feat

,Feature,importance
0,ymin,555.70
1,thickness,511.45
2,xmin,485.00
3,pixel_area,466.60
4,max_luminosity,443.40
5,xmax,434.90
6,log_area,350.20
7,ymax,341.55
8,x_component_2,110.20
9,x_component_1,110.15


In [8]:
df = X[important_feat]
corr_matrix = df.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

high_cor = [column for column in upper.columns if any(upper[column] > 0.98)]
print(len(high_cor))
print(high_cor)

4
['xmax', 'log_area', 'ymax', 'grade_A_Component_2']


In [9]:
features = [i for i in important_feat if i not in high_cor]
print(len(features))
print(features)

11
['ymin', 'thickness', 'xmin', 'pixel_area', 'max_luminosity', 'x_component_2', 'x_component_1', 'x_component_3', 'x_component_5', 'x_component_4', 'grade_A_Component_1']


In [10]:
X1 = X[features]
X_test1 = X_test[features]

In [11]:
err = []
y_pred_tot_lgm = []

from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=20, shuffle=True)
i = 1
for train_index, test_index in fold.split(X1, y):
    x_train, x_val = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMClassifier(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=500,
          eval_metric='log_loss',
          verbose=200)
    pred_y = m.predict_proba(x_val)
    print("err_lgm: ",log_loss(y_val,pred_y))
    
    err.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test1)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.209826	valid_1's binary_logloss: 0.271661
[400]	training's binary_logloss: 0.132412	valid_1's binary_logloss: 0.28355
[600]	training's binary_logloss: 0.0870625	valid_1's binary_logloss: 0.308468
Early stopping, best iteration is:
[212]	training's binary_logloss: 0.203751	valid_1's binary_logloss: 0.269933
err_lgm:  0.26993348104908077
Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.211442	valid_1's binary_logloss: 0.248468
[400]	training's binary_logloss: 0.136536	valid_1's binary_logloss: 0.237694
[600]	training's binary_logloss: 0.0890642	valid_1's binary_logloss: 0.215116
[800]	training's binary_logloss: 0.0552759	valid_1's binary_logloss: 0.2227
[1000]	training's binary_logloss: 0.0368688	valid_1's binary_logloss: 0.229986
Early stopping, best iteration is:
[600]	training's binary_logloss: 0.0890642	valid_1's binary_logloss: 0.215116
e

[400]	training's binary_logloss: 0.127815	valid_1's binary_logloss: 0.274334
[600]	training's binary_logloss: 0.0819274	valid_1's binary_logloss: 0.295587
Early stopping, best iteration is:
[241]	training's binary_logloss: 0.191136	valid_1's binary_logloss: 0.262329
err_lgm:  0.26232910120873437
Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.21355	valid_1's binary_logloss: 0.236747
[400]	training's binary_logloss: 0.134715	valid_1's binary_logloss: 0.246818
[600]	training's binary_logloss: 0.0853194	valid_1's binary_logloss: 0.253839
Early stopping, best iteration is:
[221]	training's binary_logloss: 0.205544	valid_1's binary_logloss: 0.233772
err_lgm:  0.23377211362307534
Training until validation scores don't improve for 500 rounds.
[200]	training's binary_logloss: 0.207122	valid_1's binary_logloss: 0.33772
[400]	training's binary_logloss: 0.130421	valid_1's binary_logloss: 0.338405
[600]	training's binary_logloss: 0.0841267	valid_1'

In [12]:
np.mean(err,0)

0.2843276209689739

In [13]:
from xgboost import XGBClassifier

errxgb = []
y_pred_tot_xgb = []

from sklearn.model_selection import KFold,StratifiedKFold

fold = StratifiedKFold(n_splits=20, shuffle=True)
i = 1
for train_index, test_index in fold.split(X,y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = XGBClassifier(boosting_type='gbdt',
                      max_depth=5,
                      learning_rate=0.05,
                      n_estimators=5000,
                      random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=500,
          eval_metric='logloss',
          verbose=200)
    pred_y = m.predict_proba(x_val)
    
    print("err_xgb: ",log_loss(y_val, pred_y))
    errxgb.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test)
    i = i + 1
    y_pred_tot_xgb.append(pred_test)

[0]	validation_0-logloss:0.669862	validation_1-logloss:0.67401
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 500 rounds.
[200]	validation_0-logloss:0.193712	validation_1-logloss:0.417535
[400]	validation_0-logloss:0.105472	validation_1-logloss:0.445509
Stopping. Best iteration:
[91]	validation_0-logloss:0.272745	validation_1-logloss:0.404649

err_xgb:  0.4046491611128052
[0]	validation_0-logloss:0.669674	validation_1-logloss:0.67805
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 500 rounds.
[200]	validation_0-logloss:0.196731	validation_1-logloss:0.39295
[400]	validation_0-logloss:0.106476	validation_1-logloss:0.457447
[600]	validation_0-logloss:0.065579	validation_1-logloss:0.51739
Stopping. Best iteration:
[102]	validation_0-logloss:0.261188	validation_1-logloss:0.380223



[200]	validation_0-logloss:0.201251	validation_1-logloss:0.275668
[400]	validation_0-logloss:0.11079	validation_1-logloss:0.233004
[600]	validation_0-logloss:0.068583	validation_1-logloss:0.210584
[800]	validation_0-logloss:0.049183	validation_1-logloss:0.203254
[1000]	validation_0-logloss:0.03677	validation_1-logloss:0.196985
[1200]	validation_0-logloss:0.029015	validation_1-logloss:0.205856
[1400]	validation_0-logloss:0.024529	validation_1-logloss:0.203873
Stopping. Best iteration:
[998]	validation_0-logloss:0.036856	validation_1-logloss:0.196775

err_xgb:  0.19677490171100667
[0]	validation_0-logloss:0.670018	validation_1-logloss:0.672865
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 500 rounds.
[200]	validation_0-logloss:0.204618	validation_1-logloss:0.233636
[400]	validation_0-logloss:0.107948	validation_1-logloss:0.20561
[600]	validation_0-logloss:0.066858	validation_1-logl

In [14]:
np.mean(errxgb,0)

0.2752543015588146

In [15]:
(np.mean(errxgb, 0) + np.mean(err, 0))/2

0.27979096126389424

In [16]:
submission = pd.DataFrame((np.mean(y_pred_tot_lgm, 0)+np.mean(y_pred_tot_xgb, 0))/2)
submission.columns = [1,2]
submission.to_excel('Submission.xlsx', index=False)

In [17]:
submission.head()

,1,2
0,0.995663,0.004337
1,0.196165,0.803835
2,0.991076,0.008924
3,0.995380,0.004620
4,0.754443,0.245557
